# csv
파일은 쉼표로 구분하는 구분된 텍스트 파일.

대표적으로는 엑셀형태

In [1]:
from langchain_community.document_loaders.csv_loader import CSVLoader

In [2]:
loader = CSVLoader(file_path = "./data/titanic.csv")

docs = loader.load()

In [3]:
print(len(docs))
print(docs[0].metadata)

891
{'source': './data/titanic.csv', 'row': 0}


In [4]:
print(docs[1].page_content)

PassengerId: 2
Survived: 1
Pclass: 1
Name: Cumings, Mrs. John Bradley (Florence Briggs Thayer)
Sex: female
Age: 38
SibSp: 1
Parch: 0
Ticket: PC 17599
Fare: 71.2833
Cabin: C85
Embarked: C


# CSV 파싱 및 로딩 커스터마이징
CSV파일 자체를 임베딩해서 넣어주면, LLM이 잘 인지를 안하는 경우가 많다

In [7]:
loader = CSVLoader(
    file_path = "./data/titanic.csv",
    csv_args={
        "delimiter": ',', # 구분자. tab인 경우에는 tab으로 바꿔줌
        "quotechar": '"', # 인용 부호 문자
        "fieldnames": [
            "PassengerId",
            "Survived (1: Survived, 0: Died)",
            "Passenger Class",
            "Name",
            "Sex",
            "Age",
            "Number of Siblings/Spouses Aboard",
            "Number of Parents/ Children Aboard",
            "Ticken Number",
            "Fare",
            "Cabin",
            "Port of Embarkation",
        ], #필드이름
    }
)

In [10]:
docs = loader.load()

print(docs[1].page_content)

PassengerId: 1
Survived (1: Survived, 0: Died): 0
Passenger Class: 3
Name: Braund, Mr. Owen Harris
Sex: male
Age: 22
Number of Siblings/Spouses Aboard: 1
Number of Parents/ Children Aboard: 0
Ticken Number: A/5 21171
Fare: 7.25
Cabin: 
Port of Embarkation: S


In [11]:
docs = loader.load()

print(docs[0].page_content)

PassengerId: PassengerId
Survived (1: Survived, 0: Died): Survived
Passenger Class: Pclass
Name: Name
Sex: Sex
Age: Age
Number of Siblings/Spouses Aboard: SibSp
Number of Parents/ Children Aboard: Parch
Ticken Number: Ticket
Fare: Fare
Cabin: Cabin
Port of Embarkation: Embarked


# xml문서처리해서 넣어주는게 편하다.


In [13]:
# 0번 정보는 헤더정보이기 떄문에 skip
# 이렇게하면 한행이 xml로 태그가 되어 들어간다. 검색된 결과를 passenger id로 감싸서 출력된다.
row = docs[1].page_content.split("\n")
row_str = "<row>"
for element in row:
    splitted_element = element.split(":")
    value = splitted_element[-1]
    col = ":".join(splitted_element[:-1])
    row_str += f"<{col}>{value.strip()}</{col}>"
row_str += "</row>"
print(row_str)

<row><PassengerId>1</PassengerId><Survived (1: Survived, 0: Died)>0</Survived (1: Survived, 0: Died)><Passenger Class>3</Passenger Class><Name>Braund, Mr. Owen Harris</Name><Sex>male</Sex><Age>22</Age><Number of Siblings/Spouses Aboard>1</Number of Siblings/Spouses Aboard><Number of Parents/ Children Aboard>0</Number of Parents/ Children Aboard><Ticken Number>A/5 21171</Ticken Number><Fare>7.25</Fare><Cabin></Cabin><Port of Embarkation>S</Port of Embarkation></row>


In [14]:
for doc in docs[1:]:
    row = doc.page_content.split("\n")
    row_str = "<row>"
    for element in row:
        splitted_element = element.split(":")
        value = splitted_element[-1]
        col = ":".join(splitted_element[:-1])
        row_str += f"<{col}>{value.strip()}</{col}>"
    row_str += "</row>"
    print(row_str)

<row><PassengerId>1</PassengerId><Survived (1: Survived, 0: Died)>0</Survived (1: Survived, 0: Died)><Passenger Class>3</Passenger Class><Name>Braund, Mr. Owen Harris</Name><Sex>male</Sex><Age>22</Age><Number of Siblings/Spouses Aboard>1</Number of Siblings/Spouses Aboard><Number of Parents/ Children Aboard>0</Number of Parents/ Children Aboard><Ticken Number>A/5 21171</Ticken Number><Fare>7.25</Fare><Cabin></Cabin><Port of Embarkation>S</Port of Embarkation></row>
<row><PassengerId>2</PassengerId><Survived (1: Survived, 0: Died)>1</Survived (1: Survived, 0: Died)><Passenger Class>1</Passenger Class><Name>Cumings, Mrs. John Bradley (Florence Briggs Thayer)</Name><Sex>female</Sex><Age>38</Age><Number of Siblings/Spouses Aboard>1</Number of Siblings/Spouses Aboard><Number of Parents/ Children Aboard>0</Number of Parents/ Children Aboard><Ticken Number>PC 17599</Ticken Number><Fare>71.2833</Fare><Cabin>C85</Cabin><Port of Embarkation>C</Port of Embarkation></row>
<row><PassengerId>3</Pass

source_column 인자를 사용하여 각 행에서 생성된 문서의 출처를 지정하세요. 그렇지 않으면 모든 문서의 출처로 file_path가 사용됩니다.

이는 CSV 파일에서 로드된 문서를 출처를 사용하여 질문에 답하는 체인에 사용할 때 유용합니다.


In [22]:
loader = CSVLoader(
    file_path="./data/titanic.csv", source_column="PassengerId"
)  # CSV 로더 설정, 파일 경로 및 소스 컬럼 지정

docs = loader.load()  # 데이터 로드

print(docs[1])  # 데이터 출력

page_content='PassengerId: 2
Survived: 1
Pclass: 1
Name: Cumings, Mrs. John Bradley (Florence Briggs Thayer)
Sex: female
Age: 38
SibSp: 1
Parch: 0
Ticket: PC 17599
Fare: 71.2833
Cabin: C85
Embarked: C' metadata={'source': '2', 'row': 1}


In [23]:
print(docs[0])

page_content='PassengerId: 1
Survived: 0
Pclass: 3
Name: Braund, Mr. Owen Harris
Sex: male
Age: 22
SibSp: 1
Parch: 0
Ticket: A/5 21171
Fare: 7.25
Cabin: 
Embarked: S' metadata={'source': '1', 'row': 0}


# UnstructuredCSVLoader

UnstructuredCSVLoader를 사용하여 테이블을 로드할 수도 있습니다. UnstructuredCSVLoader를 사용하는 한 가지 장점은 "elements" 모드에서 사용할 경우, 메타데이터에서 테이블의 HTML 표현이 제공된다는 것입니다.

In [17]:
from langchain_community.document_loaders.csv_loader import UnstructuredCSVLoader

# 비구조화 CSV 로더 인스턴스 생성
loader = UnstructuredCSVLoader(file_path="./data/titanic.csv", mode="elements")

# 문서 로드
docs = loader.load()

# 첫 번째 문서의 HTML 텍스트 메타데이터 출력
print(docs[0].metadata["text_as_html"][:1000])

ModuleNotFoundError: No module named 'iso639.language'

# DataFrameLoader
- Pandas는 Python 프로그래밍 언어를 위한 오픈 소스 데이터 분석 및 조작 도구입니다. 이 라이브러리는 데이터 과학, 머신러닝, 그리고 다양한 분야의 데이터 작업에 널리 사용되고 있습니다.

In [18]:
import pandas as pd

# CSV 파일 읽기
df = pd.read_csv("./data/titanic.csv")

In [19]:
# 데이터프레임의 처음 다섯 행 조회
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [20]:
from langchain_community.document_loaders import DataFrameLoader

# 데이터 프레임 로더 설정, 페이지 내용 컬럼 지정
loader = DataFrameLoader(df, page_content_column="Name")

# 문서 로드
docs = loader.load()

# 데이터 출력
print(docs[0].page_content)

# 메타데이터 출력
print(docs[0].metadata)

Braund, Mr. Owen Harris
{'PassengerId': 1, 'Survived': 0, 'Pclass': 3, 'Sex': 'male', 'Age': 22.0, 'SibSp': 1, 'Parch': 0, 'Ticket': 'A/5 21171', 'Fare': 7.25, 'Cabin': nan, 'Embarked': 'S'}


In [21]:
# 큰 테이블에 대한 지연 로딩, 전체 테이블을 메모리에 로드하지 않음
for row in loader.lazy_load():
    print(row)
    break  # 첫 행만 출력

page_content='Braund, Mr. Owen Harris' metadata={'PassengerId': 1, 'Survived': 0, 'Pclass': 3, 'Sex': 'male', 'Age': 22.0, 'SibSp': 1, 'Parch': 0, 'Ticket': 'A/5 21171', 'Fare': 7.25, 'Cabin': nan, 'Embarked': 'S'}
